<a href="https://colab.research.google.com/github/kjedrzejczak/Wprowadzenie_do_sieci_neuronowych-Tensorflow_2.0-Keras/blob/main/Przeuczenie_niedouczenia_metody_regularyzacji_II_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


sns.set()

In [20]:
NUM_WORDS = 10000   # 10000 najszęstrzych słów
INDEX_FROM = 3

(train_data, train_lables), (test_data, test_lables) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

In [3]:
print(train_data.shape)
print(train_lables.shape)
print(test_data.shape)
print(test_lables.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [4]:
train_lables[0]

1

In [5]:
print(train_data[0])  # częstośc występowania słowa w recenzjach np. 14 - słowo wystąpiło na  14 miejscu w recenzjach

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [6]:
np.unique(train_lables)

array([0, 1])

In [21]:
# tłumaczenie słów na tekst przy pomocy funkcji get_word_index()
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<start>'] = 1
word_to_idx['<UKN>'] = 2
word_to_idx['<UNUSED'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))

<start> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UKN> is an amazing actor and now the same being director <UKN> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UKN> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UKN> to the two little boy's that played the <UKN> of norman and paul they were just brilliant children are often left out of the <UKN> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [8]:
train_lables[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [22]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [10]:
test_data.shape

(25000, 10000)

# Budowanie modelu

In [13]:
baseline_model = Sequential()
baseline_model.add(Dense(units=16, activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
baseline_model.add(Dense(units=16, activation='relu'))
baseline_model.add(Dense(units=1, activation='sigmoid'))  # blasifikacja binarna

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                160016    
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
baseline_history = baseline_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 5s - loss: 0.4872 - accuracy: 0.8099 - binary_crossentropy: 0.4872 - val_loss: 0.3409 - val_accuracy: 0.8755 - val_binary_crossentropy: 0.3409 - 5s/epoch - 96ms/step
Epoch 2/20
49/49 - 2s - loss: 0.2514 - accuracy: 0.9104 - binary_crossentropy: 0.2514 - val_loss: 0.2829 - val_accuracy: 0.8886 - val_binary_crossentropy: 0.2829 - 2s/epoch - 36ms/step
Epoch 3/20
49/49 - 2s - loss: 0.1838 - accuracy: 0.9339 - binary_crossentropy: 0.1838 - val_loss: 0.2927 - val_accuracy: 0.8846 - val_binary_crossentropy: 0.2927 - 2s/epoch - 50ms/step
Epoch 4/20
49/49 - 2s - loss: 0.1463 - accuracy: 0.9507 - binary_crossentropy: 0.1463 - val_loss: 0.3107 - val_accuracy: 0.8800 - val_binary_crossentropy: 0.3107 - 2s/epoch - 47ms/step
Epoch 5/20
49/49 - 2s - loss: 0.1189 - accuracy: 0.9613 - binary_crossentropy: 0.1189 - val_loss: 0.3386 - val_accuracy: 0.8752 - val_binary_crossentropy: 0.3386 - 2s/epoch - 35ms/step
Epoch 6/20
49/49 - 2s - loss: 0.0973 - accuracy: 0.9700 - binary_crossentro

# Budowa mniejszego modelu

In [13]:
smaller_model = Sequential()
smaller_model.add(Dense(units=4, activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
smaller_model.add(Dense(units=4, activation='relu'))
smaller_model.add(Dense(units=1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 4)                 40004     
                                                                 
 dense_4 (Dense)             (None, 4)                 20        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 40029 (156.36 KB)
Trainable params: 40029 (156.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
smaller_history = smaller_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 6s - loss: 0.6385 - accuracy: 0.6252 - binary_crossentropy: 0.6385 - val_loss: 0.5896 - val_accuracy: 0.7154 - val_binary_crossentropy: 0.5896 - 6s/epoch - 132ms/step
Epoch 2/20
49/49 - 2s - loss: 0.5454 - accuracy: 0.7842 - binary_crossentropy: 0.5454 - val_loss: 0.5321 - val_accuracy: 0.7841 - val_binary_crossentropy: 0.5321 - 2s/epoch - 35ms/step
Epoch 3/20
49/49 - 2s - loss: 0.4904 - accuracy: 0.8474 - binary_crossentropy: 0.4904 - val_loss: 0.4961 - val_accuracy: 0.8195 - val_binary_crossentropy: 0.4961 - 2s/epoch - 34ms/step
Epoch 4/20
49/49 - 2s - loss: 0.4513 - accuracy: 0.8794 - binary_crossentropy: 0.4513 - val_loss: 0.4695 - val_accuracy: 0.8525 - val_binary_crossentropy: 0.4695 - 2s/epoch - 33ms/step
Epoch 5/20
49/49 - 2s - loss: 0.4208 - accuracy: 0.9010 - binary_crossentropy: 0.4208 - val_loss: 0.4510 - val_accuracy: 0.8597 - val_binary_crossentropy: 0.4510 - 2s/epoch - 34ms/step
Epoch 6/20
49/49 - 3s - loss: 0.3952 - accuracy: 0.9148 - binary_crossentr

# Budowa większego modelu

In [15]:
bigger_model = Sequential()
bigger_model.add(Dense(units=256, activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
bigger_model.add(Dense(units=256, activation='relu'))
bigger_model.add(Dense(units=1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               2560256   
                                                                 
 dense_7 (Dense)             (None, 256)               65792     
                                                                 
 dense_8 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2626305 (10.02 MB)
Trainable params: 2626305 (10.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
bigger_history = bigger_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 16s - loss: 0.3573 - accuracy: 0.8500 - binary_crossentropy: 0.3573 - val_loss: 0.2961 - val_accuracy: 0.8781 - val_binary_crossentropy: 0.2961 - 16s/epoch - 337ms/step
Epoch 2/20
49/49 - 11s - loss: 0.1664 - accuracy: 0.9378 - binary_crossentropy: 0.1664 - val_loss: 0.3291 - val_accuracy: 0.8724 - val_binary_crossentropy: 0.3291 - 11s/epoch - 231ms/step
Epoch 3/20
49/49 - 9s - loss: 0.0897 - accuracy: 0.9700 - binary_crossentropy: 0.0897 - val_loss: 0.3968 - val_accuracy: 0.8657 - val_binary_crossentropy: 0.3968 - 9s/epoch - 179ms/step
Epoch 4/20
49/49 - 12s - loss: 0.0302 - accuracy: 0.9924 - binary_crossentropy: 0.0302 - val_loss: 0.5135 - val_accuracy: 0.8663 - val_binary_crossentropy: 0.5135 - 12s/epoch - 255ms/step
Epoch 5/20
49/49 - 13s - loss: 0.0067 - accuracy: 0.9993 - binary_crossentropy: 0.0067 - val_loss: 0.6323 - val_accuracy: 0.8646 - val_binary_crossentropy: 0.6323 - 13s/epoch - 257ms/step
Epoch 6/20
49/49 - 13s - loss: 0.0012 - accuracy: 1.0000 - bin

In [17]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,loss,accuracy,binary_crossentropy,val_loss,val_accuracy,val_binary_crossentropy,epoch
0,0.496844,0.80304,0.496844,0.344877,0.87544,0.344877,0
1,0.254715,0.90960,0.254715,0.284661,0.88836,0.284661,1
2,0.185512,0.93476,0.185512,0.286574,0.88748,0.286574,2
3,0.148109,0.94972,0.148109,0.304881,0.88220,0.304881,3
4,0.120629,0.96076,0.120629,0.333205,0.87752,0.333205,4


In [18]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

## val_binary_crossentropy - najważniejszy - trzeba znaleśc minimum
Najlepszy jest model najmniejszy i należy go ograniczyć do tylko 5 epok !!!!

# Metody regularyzacji

In [11]:
from tensorflow.keras.regularizers import l2  # 👈👈👈👈👈👈👈

l2_model = Sequential()
l2_model.add(Dense(units=16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))   # NUM_WORDS = 10_000
l2_model.add(Dense(units=16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(units=1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                160016    
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
l2_history = l2_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables), verbose=2)

Epoch 1/20
49/49 - 6s - loss: 0.6671 - accuracy: 0.8034 - binary_crossentropy: 0.5115 - val_loss: 0.5168 - val_accuracy: 0.8697 - val_binary_crossentropy: 0.3698 - 6s/epoch - 115ms/step
Epoch 2/20
49/49 - 2s - loss: 0.4323 - accuracy: 0.8992 - binary_crossentropy: 0.2873 - val_loss: 0.4455 - val_accuracy: 0.8816 - val_binary_crossentropy: 0.3049 - 2s/epoch - 37ms/step
Epoch 3/20
49/49 - 2s - loss: 0.3679 - accuracy: 0.9167 - binary_crossentropy: 0.2315 - val_loss: 0.4221 - val_accuracy: 0.8836 - val_binary_crossentropy: 0.2907 - 2s/epoch - 50ms/step
Epoch 4/20
49/49 - 3s - loss: 0.3324 - accuracy: 0.9292 - binary_crossentropy: 0.2047 - val_loss: 0.4096 - val_accuracy: 0.8841 - val_binary_crossentropy: 0.2863 - 3s/epoch - 59ms/step
Epoch 5/20
49/49 - 2s - loss: 0.3078 - accuracy: 0.9366 - binary_crossentropy: 0.1874 - val_loss: 0.4027 - val_accuracy: 0.8839 - val_binary_crossentropy: 0.2862 - 2s/epoch - 36ms/step
Epoch 6/20
49/49 - 2s - loss: 0.2911 - accuracy: 0.9408 - binary_crossentr

In [17]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [18]:

from tensorflow.keras.layers import Dropout  # porzucz częśc neuronów w trenowaniu modelu  # 👈👈👈👈👈👈👈

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                160016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160305 (626.19 KB)
Trainable params: 160305 (626.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
dropout_history = dropout_model.fit(train_data, train_lables, epochs=20, batch_size=512, validation_data=(test_data, test_lables))

Epoch 1/20
49/49 [==============================] - 6s 102ms/step - loss: 0.6589 - accuracy: 0.5954 - binary_crossentropy: 0.6589 - val_loss: 0.5770 - val_accuracy: 0.8148 - val_binary_crossentropy: 0.5770
Epoch 2/20
49/49 [==============================] - 2s 41ms/step - loss: 0.5340 - accuracy: 0.7845 - binary_crossentropy: 0.5340 - val_loss: 0.4384 - val_accuracy: 0.8635 - val_binary_crossentropy: 0.4384
Epoch 3/20
49/49 [==============================] - 2s 39ms/step - loss: 0.4346 - accuracy: 0.8456 - binary_crossentropy: 0.4346 - val_loss: 0.3597 - val_accuracy: 0.8851 - val_binary_crossentropy: 0.3597
Epoch 4/20
49/49 [==============================] - 2s 40ms/step - loss: 0.3653 - accuracy: 0.8774 - binary_crossentropy: 0.3653 - val_loss: 0.3183 - val_accuracy: 0.8881 - val_binary_crossentropy: 0.3183
Epoch 5/20
49/49 [==============================] - 2s 42ms/step - loss: 0.3197 - accuracy: 0.8978 - binary_crossentropy: 0.3197 - val_loss: 0.2960 - val_accuracy: 0.8891 - val_bi

In [26]:

fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()